In [6]:
%pip install -Uq torch accelerate

Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "yanolja/YanoljaNEXT-Rosetta-4B"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    dtype=torch.bfloat16,
    device_map="auto",
    max_memory={0: "23GB"},
)

/Users/anpigon/.pyenv/versions/3.12.11/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Device 0 is not available, available devices are []
Device 0 is not available, available devices are []
Loading checkpoint shards: 100%|██████████| 3/3 [00:06<00:00,  2.19s/it]


In [4]:
model.device

device(type='mps', index=0)

In [ ]:
target_language = "Korean"
messages = [
    {
        "role": "system",
        "content": (
            f"Translate the user's text to {target_language}."
            "Output only one sentence of the final translation"
            "Do not print explanations, thought processes, or examples."
        ),
    },
    {
        "role": "user",
        "content": "Yanolja NEXT is a company that provides global cutting-edge technology for the hospitality industry.",
    },
]
prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
input_length = inputs["input_ids"].shape[1]

with torch.inference_mode():
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=False,
        eos_token_id=tokenizer.eos_token_id,
    )

generated_tokens = outputs[0][input_length:]
translation = tokenizer.decode(generated_tokens, skip_special_tokens=True)


The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [3]:
translation

'요날자 NEXT는 숙박 산업을 위한 글로벌 최첨단 기술을 제공하는 회사입니다.'

"instruction\nTranslate the user's text to Korean.Output only one sentence of the final translationDo not print explanations, thought processes, or examples.\nsource\nYanolja NEXT is a company that provides global cutting-edge technology for the hospitality industry.\ntranslation\n요날자 NEXT는 숙박 산업을 위한 글로벌 최첨단 기술을 제공하는 회사입니다."